In [0]:
%scala
display(dbutils.fs.ls("/mnt/storage41623/raw/donuts"))

In [0]:
%scala
import scala.io.Source

val src = Source.fromFile("/dbfs/mnt/storage41623/raw/donuts/donut7-bad.json", "UTF-8")
val jContent = try src.mkString finally src.close()

import scala.io.Source
src: scala.io.BufferedSource = <iterator>
jContent: String =
{
 "id": "0007",
 "type": true,
 "name": "Lemon Cake",
 "ppu": 0.45,
 "batters":
 "batter": [
 {
 "id": "1001",
 "type": "Regular"
 },
 {
 "id": "1002",
 "type": "Chocolate"
 },
 {
 "id": "1003",
 "type": "Blueberry"
 },
 {
 "id": "1004",
 "type": "Devil's Food"
 }
 ]
 },
 "topping": [
 {
 "id": "5001",
 "type": "None"
 },
 {
 "id": "5002",
 "type": "Glazed"
 },
 {
 "id": "5005",
 "type": "Sugar"
 },
 {
 "id": "5007",
 "type": "Powdered Sugar"
 },
 {
 "id": "5006",
 "type": "Chocolate with Sprinkles"
 },
 {
 "id": "5003",
 "type": "Chocolate"
 },
 {
 "id": "5004",
 "type": "Maple"
 }
 ]
}

In [0]:
%scala
import org.json.{JSONObject, JSONException}

def isValidJSON(json: String): Boolean = {
  try {
    new JSONObject(json)
  } catch {
    case e: JSONException => {
      println("<jfile_name> is not valid JSON")
      return false
    }
  }
    return true
}

import org.json.{JSONObject, JSONException}
isValidJSON: (json: String)Boolean

In [0]:
%scala
isValidJSON(jContent)

<jfile_name> is not valid JSON
res5: Boolean = false

In [0]:
%scala
def mult(a: Int, b: Int): Int = {
  a * b
}

println(mult(2,4))

8
mult: (a: Int, b: Int)Int

In [0]:
%scala
import org.apache.spark.sql.Row
import org.apache.spark.sql.types.{StructType, StructField, StringType}

val data = List(Row("1", jContent))

val dfSchema = StructType(fields = Seq(
  StructField("id", StringType, false),
  StructField("json", StringType, false)
))


val df = spark.createDataFrame(sc.parallelize(data), StructType(dfSchema))


In [0]:
%scala
display(df)

In [0]:
%scala
import org.apache.spark.sql.functions.{col, get_json_object}

// df.select(get_json_object(col("json"),"$.ppu").as("ppu")).show(false)
df.select(get_json_object(col("json"),"$.ppu")).head().getString(0)


In [0]:
%scala
val jPathMap = Map(
  "$.id" -> "dnt.id",
  "$.type" -> "dnt.type",
  "$.ppu" -> "dnt.ppu",
  "$.batters.batter[:]" -> "dnt.batters",
  "$.topping" -> "dnt.toppings"
)

var data = Seq.empty[String]

jPathMap.foreach { case (jpath, colm) =>
  data = data :+ df.select(get_json_object(col("json"), jpath)).head().getString(0)
}

In [0]:
%scala
import scala.collection.mutable.{Map => MutableMap}
import scala.collection.immutable.{ListMap}

val jPathMap = ListMap(
  "$.id" -> "dnt_id",
  "$.type" -> "dnt_type",
  "$.ppu" -> "dnt_ppu",
  "$.batters.batter" -> "dnt_batters",  // check null value case
  "$.topping[:]" -> "dnt_toppings"
)

var rowData = MutableMap.empty[String, String]

jPathMap.foreach { case (jpath, colm) =>
  rowData += (colm -> df.select(get_json_object(col("json"), jpath)).head().getString(0))
}

In [0]:
%scala
import org.apache.spark.util.SizeEstimator

SizeEstimator.estimate(jPathMap)
SizeEstimator.estimate(rowData)

In [0]:
%scala
import org.apache.spark.sql.Row
import org.apache.spark.sql.types.{StringType, StructField, StructType}

val schema = StructType(rowData.keys.map(StructField(_, StringType, true)).toSeq)
val data = Seq(Row(schema.fieldNames.map(n => rowData(n)).toSeq:_*))
// val data = Seq(Row(rowData.values.toSeq:_*))

val df2 = spark.createDataFrame(sc.parallelize(data), schema)

df2.show()

In [0]:
%scala
import org.json.JSONObject

In [0]:
%scala
Row(schema.fieldNames.map( n => rowData(n) ).toSeq:_*)

In [0]:
%scala

var ss = Seq()
var ds = Seq.empty[String]

rowData.foreach { case (colm, value) =>
  
  // println(colm + ":" + value)
}

In [0]:
%scala
import io.delta.tables.{DeltaTable}

// val targetDT = DeltaTable.forPath(spark, "/mnt/formula1dl/demo/drivers_merge")
val targetDT = DeltaTable.forName("db1.donuts")

// val targetDf = spark.read.format("delta").load("/mnt/storage41623/processed/donuts")
// val targetDf = spark.read.format("delta").table("db1.donuts")

In [0]:
%scala
targetDT
  .as("target")
  .merge(df2.as("source"), "source.dnt_id = target.dnt_id")
  .whenMatched()
  .updateAll()
  .whenNotMatched()
  .insertAll()
  .execute()

In [0]:
%sql
select *
  from db1.donuts; 